In [1]:
import pandas as pd      
import numpy as np 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import xgboost


In [373]:
#pip list

In [374]:
#pip install streamlit==1.10.0

In [375]:
#pip install scikit-learn==1.0.2

In [2]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    score = r2_score(actual, pred)
    return print(" r2_score:", score, "\n","mae:", mae, "\n","mse:",mse, "\n","rmse:",rmse)

In [22]:
df=pd.read_csv("final_scout_not_dummy.csv")
pd.set_option('display.max_columns',  None)
pd.options.display.float_format = '{:,.1f}'.format

In [4]:
df.head(1)

,make_model,body_type,price,vat,km,Type,Fuel,Gears,Comfort_Convenience,Entertainment_Media,Extras,Safety_Security,age,Previous_Owners,hp_kW,Inspection_new,Paint_Type,Upholstery_type,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb
0,Audi A1,Sedans,15770,VAT deductible,"56,013.0",Used,Diesel,7.0,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...",3.0,2.0,66.0,1,Metallic,Cloth,Automatic,"1,422.0","1,220.0",front,3.8


In [379]:
df.shape

(15915, 23)

In [380]:
df.isnull().sum().any()

False

In [381]:
df.columns

Index(['make_model', 'body_type', 'price', 'vat', 'km', 'Type', 'Fuel',
       'Gears', 'Comfort_Convenience', 'Entertainment_Media', 'Extras',
       'Safety_Security', 'age', 'Previous_Owners', 'hp_kW', 'Inspection_new',
       'Paint_Type', 'Upholstery_type', 'Gearing_Type', 'Displacement_cc',
       'Weight_kg', 'Drive_chain', 'cons_comb'],
      dtype='object')

In [382]:
df.age.unique()

array([3., 2., 1., 0.])

In [383]:
df.hp_kW.min()

40.0

In [384]:
df.hp_kW.max()

294.0

In [385]:
df.km.nunique()

6691

In [386]:
df.km.max()

317000.0

In [387]:
df.km.min()

0.0

In [388]:
df.Gearing_Type.unique()

array(['Automatic', 'Manual', 'Semi-automatic'], dtype=object)

In [23]:
df.make_model.unique()

array(['Audi A1', 'Audi A2', 'Audi A3', 'Opel Astra', 'Opel Corsa',
       'Opel Insignia', 'Renault Clio', 'Renault Duster',
       'Renault Espace'], dtype=object)

In [24]:
df.make_model.value_counts()

Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2525
Opel Corsa        2216
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: make_model, dtype: int64

In [25]:
df[df.make_model=="Audi A2"]

,make_model,body_type,price,vat,km,Type,Fuel,Gears,Comfort_Convenience,Entertainment_Media,Extras,Safety_Security,age,Previous_Owners,hp_kW,Inspection_new,Paint_Type,Upholstery_type,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb
2614,Audi A2,Off-Road,28200,VAT deductible,"26,166.0",Employee's car,Diesel,6.0,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...",Alloy wheels,"ABS,Adaptive Cruise Control,Central door lock,...",1.0,1.0,85.0,0,Metallic,Cloth,Manual,"1,598.0","1,135.0",front,4.9


In [38]:
df.drop(index=[2614], inplace =True)

In [39]:
df.make_model.unique()

array(['Audi A1', 'Audi A3', 'Opel Astra', 'Opel Corsa', 'Opel Insignia',
       'Renault Clio', 'Renault Duster', 'Renault Espace'], dtype=object)

In [40]:
new_list=["age", "hp_kW", "km", "Gearing_Type", 'make_model']

In [41]:
X=df[new_list]
y=df['price']

In [42]:
X.sample(5)

,age,hp_kW,km,Gearing_Type,make_model
11229,3.0,88.0,"31,665.0",Manual,Opel Insignia
13601,2.0,55.0,"45,206.0",Manual,Renault Clio
14869,0.0,56.0,5.0,Manual,Renault Clio
5287,0.0,85.0,28.0,Automatic,Audi A3
7642,0.0,147.0,5.0,Automatic,Opel Astra


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [44]:
from sklearn.compose import make_column_transformer

In [9]:
from sklearn.preprocessing import OrdinalEncoder

In [45]:
cat = X_train.select_dtypes("object").columns
cat

Index(['Gearing_Type', 'make_model'], dtype='object')

In [46]:
ord_enc = OrdinalEncoder()
column_trans = make_column_transformer((ord_enc, cat), remainder='passthrough') # remainder: # Keep other columns rather than cat in original form.
                                                                                # remainder: # if you want to scale rest of cat columns use "minmax" istead passthroug
    
    # Apply ord_enc to categoric variables and remain in original form rest of them.
    # ord_enc for DT based algorithms
    # For Linear reg, log reg vs. use one hot encoder 

In [47]:
X_train=column_trans.fit_transform(X_train)

In [48]:
X_test=column_trans.transform(X_test)

In [49]:
X_train

array([[1.00e+00, 5.00e+00, 0.00e+00, 5.60e+01, 1.00e+01],
       [1.00e+00, 2.00e+00, 3.00e+00, 8.10e+01, 1.39e+05],
       [0.00e+00, 1.00e+00, 0.00e+00, 8.50e+01, 1.80e+04],
       ...,
       [0.00e+00, 3.00e+00, 0.00e+00, 6.60e+01, 3.55e+02],
       [0.00e+00, 4.00e+00, 3.00e+00, 1.25e+02, 2.32e+05],
       [0.00e+00, 1.00e+00, 3.00e+00, 8.10e+01, 9.70e+04]])

In [405]:
pd.Series(X_train[1])

0         1.0
1         2.0
2         3.0
3        81.0
4   139,000.0
dtype: float64

In [50]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [51]:
y_pred = rf_model.predict(X_test)

In [52]:
eval_metrics(y_test,y_pred)

 r2_score: 0.9298667276611129 
 mae: 1133.7807856506083 
 mse: 3511460.4738753866 
 rmse: 1873.8891306252317


In [53]:
import pickle
pickle.dump(rf_model, open('rf_model_new', 'wb'))

In [54]:
pickle.dump(column_trans, open('transformer', 'wb'))

**deployment**

In [55]:
import pickle
kopya_model = pickle.load(open('rf_model_new', 'rb'))

In [56]:
kopya_transformer = pickle.load(open('transformer', 'rb'))

In [57]:
columns=list(X.columns)
columns

['age', 'hp_kW', 'km', 'Gearing_Type', 'make_model']

In [58]:
my_dict = {
    "age": 2,
    "hp_kW": 105,
    "km": 100000,
    "Gearing_Type":"Automatic",
    "make_model": 'Audi A3'
}

In [415]:
#df = pd.DataFrame([my_dict])
#df = pd.DataFrame(my_dict, index=["a"])

In [59]:
df = pd.DataFrame.from_dict([my_dict])

In [60]:
df.head()

,age,hp_kW,km,Gearing_Type,make_model
0,2,105,100000,Automatic,Audi A3


In [61]:
df2 = kopya_transformer.transform(df)

In [62]:
pd.Series(df2[0])

0         0.0
1         1.0
2         2.0
3       105.0
4   100,000.0
dtype: float64

In [63]:
prediction = kopya_model.predict(df2)
print(prediction)

[19269.94]


In [64]:
print("The estimated price of your car is €{}. ".format(int(prediction[0])))

The estimated price of your car is €19269. 


In [68]:
pickle.dump(kopya_model, open("rf_model_new", "wb"))